In [3]:
import warnings
import tensorflow as tf
from tensorflow.python.util.tf_export import keras_export
warnings.filterwarnings("ignore")

In [4]:
k=3
b=2
data = tf.random.normal((b,10))
_,tops = tf.nn.top_k(data,k)

batch_size = tf.constant([b])
batch_ids = tf.range(0, batch_size, dtype=tf.int32)
batch_ids = tf.tile(batch_ids[:, None], (1, k), name="batch_ids")
tops.shape,batch_ids.shape

(TensorShape([2, 3]), TensorShape([2, 3]))

In [5]:
tops,batch_ids

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[6, 8, 4],
        [5, 2, 8]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[0, 0, 0],
        [1, 1, 1]], dtype=int32)>)

In [6]:
idx = tf.stack([batch_ids, tops], axis=-1) 
idx

<tf.Tensor: shape=(2, 3, 2), dtype=int32, numpy=
array([[[0, 6],
        [0, 8],
        [0, 4]],

       [[1, 5],
        [1, 2],
        [1, 8]]], dtype=int32)>

In [7]:
idx = tf.reshape(idx, (-1, 2), name="indices")
idx

<tf.Tensor: shape=(6, 2), dtype=int32, numpy=
array([[0, 6],
       [0, 8],
       [0, 4],
       [1, 5],
       [1, 2],
       [1, 8]], dtype=int32)>

In [8]:
result = tf.SparseTensor(tf.cast(idx, tf.int64), tf.ones(batch_size * k, dtype=tf.float32),
                                 tf.shape(data, out_type=tf.int64))


In [9]:
tf.sparse.reorder(result)

SparseTensor(indices=tf.Tensor(
[[0 4]
 [0 6]
 [0 8]
 [1 2]
 [1 5]
 [1 8]], shape=(6, 2), dtype=int64), values=tf.Tensor([1. 1. 1. 1. 1. 1.], shape=(6,), dtype=float32), dense_shape=tf.Tensor([ 2 10], shape=(2,), dtype=int64))

In [17]:
a,b = result.shape
a,b

(2, 10)

In [44]:
index_tuples = tf.constant([[0,1],[0,7],[1,1],[1,9]])
y_true = tf.SparseTensor(tf.cast(index_tuples,tf.int64),tf.ones(4,dtype=tf.float32),tf.shape(data, out_type=tf.int64))
y_true

SparseTensor(indices=tf.Tensor(
[[0 1]
 [0 7]
 [1 1]
 [1 9]], shape=(4, 2), dtype=int64), values=tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([ 2 10], shape=(2,), dtype=int64))

In [50]:
result_expanded = tf.sparse.add(result,y_true.with_values(tf.zeros_like(y_true.values)))
result_expanded

SparseTensor(indices=tf.Tensor(
[[0 1]
 [0 7]
 [0 9]
 [0 0]
 [0 7]
 [1 1]
 [1 2]
 [1 5]
 [1 3]
 [1 9]], shape=(10, 2), dtype=int64), values=tf.Tensor([0. 0. 1. 1. 1. 0. 1. 1. 1. 0.], shape=(10,), dtype=float32), dense_shape=tf.Tensor([ 2 10], shape=(2,), dtype=int64))

In [51]:
y_true_expanded = tf.sparse.add(y_true,result.with_values(tf.zeros_like(result.values)))
y_true_expanded

SparseTensor(indices=tf.Tensor(
[[0 1]
 [0 7]
 [0 9]
 [0 0]
 [0 7]
 [1 1]
 [1 2]
 [1 5]
 [1 3]
 [1 9]], shape=(10, 2), dtype=int64), values=tf.Tensor([1. 1. 0. 0. 0. 1. 0. 0. 0. 1.], shape=(10,), dtype=float32), dense_shape=tf.Tensor([ 2 10], shape=(2,), dtype=int64))

In [52]:
is_correct = y_true_expanded.with_values(result_expanded.values * y_true_expanded.values)
is_correct

SparseTensor(indices=tf.Tensor(
[[0 1]
 [0 7]
 [0 9]
 [0 0]
 [0 7]
 [1 1]
 [1 2]
 [1 5]
 [1 3]
 [1 9]], shape=(10, 2), dtype=int64), values=tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32), dense_shape=tf.Tensor([ 2 10], shape=(2,), dtype=int64))

In [56]:
total_correct = tf.sparse.reduce_sum(is_correct, axis=1)
total_correct

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [59]:
result

SparseTensor(indices=tf.Tensor(
[[0 9]
 [0 0]
 [0 7]
 [1 2]
 [1 5]
 [1 3]], shape=(6, 2), dtype=int64), values=tf.Tensor([1. 1. 1. 1. 1. 1.], shape=(6,), dtype=float32), dense_shape=tf.Tensor([ 2 10], shape=(2,), dtype=int64))

In [65]:
indices = result.indices.numpy()
values = result.values.numpy()
indices


array([[0, 9],
       [0, 0],
       [0, 7],
       [1, 2],
       [1, 5],
       [1, 3]])

In [4]:
import numpy as np
from scipy.sparse import csr_matrix

row = np.array([0,0,0,1,1,1])
col = np.array([0,7,9,2,3,5])
values = np.array([1,1,1,1,1,1])
label_matrix = csr_matrix((values, (row, col)), shape=(2, 10)).toarray()

In [6]:
import xclib.evaluation.xc_metrics as xc_metrics

In [7]:
xc_metrics.compute_inv_propesity(label_matrix,0.5,0.4)

array([0.69314718, 0.42593094, 0.69314718, 0.69314718, 0.42593094,
       0.69314718, 0.42593094, 0.69314718, 0.42593094, 0.69314718])